### set up file path for idf in eppy
(in python 3 environment)

In [ ]:
# set up file path for idf in eppy
from eppy.modeleditor import IDF
IDF.setiddname('/Applications/EnergyPlus-9-1-0/Energy+.idd')
idf= IDF('/Applications/EnergyPlus-9-1-0/ExampleFiles/VentilatedSlab.idf')

In [ ]:
idf.epw = 'USA_CO_Golden-NREL.724666_TMY3.epw'

### Edit Material

In [ ]:
# view all materials from a idf file, note ALL CAP 'MATERIAL'
materials=idf.idfobjects['MATERIAL']
materials

In [ ]:
# check numbers of materials in the idf file
len(materials)

In [ ]:
# show no.1-3 material in the list
print(materials[0:3])

In [ ]:
# view 1st material obejct from the material list
material0=idf.idfobjects['MATERIAL'][0]
material0

In [ ]:
# We could modify all the fields in this material
material0.Name='WD10_CHANGED'
material0.Specific_Heat=2000.0
material0.Thickness=0.2
material0.Conductivity=0.5
material0.Density=400
material0.Specific_Heat=1000
material0

In [ ]:
# create a new material
# !!! since idf.newidfobject is a function to idf file, avoid duplication run
newmaterial=idf.newidfobject('MATERIAL', Name="G01a 19mm gypsum board",
                            Roughness="MediumSmooth",
                            Thickness=0.019,
                            Conductivity=0.16,
                            Density=800,
                            Specific_Heat=1090)
newmaterial

In [ ]:
# check whether the newmaterial is added to the material list
len(materials)

In [ ]:
# view the last material in the list
idf.idfobjects['MATERIAL'][-1]

In [ ]:
# duplicate material 
newmaterial_copy=idf.copyidfobject(newmaterial)
newmaterial_copy

In [ ]:
# now there should be 18+1 materials in the list
len(materials)

In [ ]:
# remove material
# pop, show the removed material
# pop: popidfobject -> give it the idf key: “MATERIAL”, and the index number
# IDF.popidfobject(key, index)
idf.popidfobject('MATERIAL',-1)

In [ ]:
# now there should be 19-1 materials in the list
len(materials)

In [ ]:
# you  could also use removeidfobject -> give it the idf object to be deleted
material_toremove=idf.idfobjects['MATERIAL'][-1]
idf.removeidfobject(material_toremove)

In [ ]:
# now there should be 18-1 materials in the list
len(materials)

In [ ]:
# print all names of materials
for material in materials:
    print (material.Name)

We could use python's **for** logic here to show only material thicker than 0.1

In [ ]:
# show all material name of which thickness >0.1
[material.Name for material in materials if material.Thickness > 0.1]

## construction

In [ ]:
allconstruction=idf.idfobjects['Construction'.upper()]
allconstruction

Similarly, we could check the number of  construction by using **len()**

In [ ]:
len(allconstruction)

In [ ]:
# view construction of  wall
construction_wall=allconstruction[1]
construction_wall

We could duplicate this  construction

In [ ]:
# copy the construction WALL-1
construction_wall_duplicate=idf.copyidfobject(construction_wall)

In [ ]:
len(allconstruction)

In [ ]:
# edit the duplicated WALL-1
construction_wall_duplicate.Name='WALL-2'
construction_wall_duplicate.Outside_Layer='WD01'
construction_wall_duplicate

## building surfaces

In [ ]:
# view all Building Surfaces
allsurfaces=idf.idfobjects['BuildingSurface:Detailed'.upper()]
allsurfaces

In [ ]:
#view 1st surface in the list
surface0=allsurfaces[0]
surface0

In [ ]:
# change construction of a surface
surface0.Construction_Name='WALL-2'
surface0

## building

In [ ]:
#view building in the idf file
building=idf.idfobjects['BUILDING']
building

In [ ]:
# view type of building
print(type(building))

In [ ]:
building=idf.idfobjects['BUILDING'][0]
building

In [ ]:
# view type of building again
print(type(building))

In [ ]:
# get all field names of the building idf object
building.fieldnames

In [ ]:
# get range  of "Loads_Convergence_Tolerance_Value"
building.getrange("Loads_Convergence_Tolerance_Value")

In [ ]:
# assign a value to this field
building.Loads_Convergence_Tolerance_Value=0.6

In [ ]:
# check range of this field
building.checkrange("Loads_Convergence_Tolerance_Value")

In [ ]:
# get all field names AND value of the building idf object
for fieldname in building.fieldnames:
    print ("%s = %s" % (fieldname, building[fieldname]))

In [ ]:
# mark out of range value
from eppy.bunch_subclass import RangeError
for fieldname in building.fieldnames:
    try:
        building.checkrange(fieldname)
        print ("%s = %s #-in range" % (fieldname, building[fieldname]))
    except RangeError:
        print ("%s = %s #-****OUT OF RANGE****" % (fieldname, building[fieldname]))

In [ ]:
# change the value of load converangence tolerance back to 0.5
building.Loads_Convergence_Tolerance_Value=0.5

## edit idf with json in eppy

In [ ]:
import eppy.json_functions as json_functions

In [ ]:
## using json to update idf file
json_str = {"idf.VERSION..Version_Identifier":9.2,
            "idf.SIMULATIONCONTROL..Do_Plant_Sizing_Calculation": "No",
            "idf.BUILDING.Building.North_Axis": 52,
            }
json_functions.updateidf(idf,json_str)

In [ ]:
# view the type of json_str
print(type(json_str))

In [ ]:
#view the idf after modified
idf.printidf()

## zone


In [ ]:
#  show all zones
allzone=idf.idfobjects["zone".upper()]
allzone

In [ ]:
# view first zone in the list 
zone0=allzone[0]
zone0

In [ ]:
# display zone1 volume and area
from eppy import modeleditor
print ("volume = %s" % (zone0.Volume, ))
area1 = modeleditor.zonearea(idf, zone0.Name)
print ('area = %s ' % (area1,))

## run idf

In [ ]:
idf.run()
#idf.save()
#idf.saveas()